In [7]:
# -*- coding: utf-8 -*-

# 01-BU

"""

Describe the business objectives here

"""

# 02-DU

# Load Dataset

file = 'WA_Fn-UseC_-Telco-Customer-Churn.xlsx'

from pyspark.sql import SparkSession
# Create a Spark session
spark = SparkSession.builder \
    .appName("ExcelFileReader") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

# Read the Excel file
df = spark.read \
    .format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file)

# Show the first few rows and the shape of the DataFrame
df.show()
print((df.count(), len(df.columns)))



+----------+------+-------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|ZipCode|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female| 4584.0|          

In [8]:

df = pd.read_excel(file)
print(df.head())
df.shape
df.info()
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Explore Data

df_desc=df.describe()
round(df_desc,2)
round(df_desc.transpose(),2)

df['InternetService'].describe()

internet_service_counts = df['InternetService'].value_counts()
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))

plt.pie(internet_service_counts, labels=internet_service_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Internet Service Distribution')
plt.show()


df['MonthlyCharges'].plot.hist(bins=50)
plt.xlabel('Monthly Charges')
plt.ylabel('Frequency')
plt.title('Distribution of Monthly Charges')


df['TotalCharges'].plot.hist(bins=50)
plt.xlabel('Total Charges')
plt.ylabel('Frequency')
plt.title('Distribution of Total Charges')

df['tenure'].plot.hist(bins=50)
plt.xlabel('Tenure')
plt.ylabel('Frequency')
plt.title('Distribution of Tenure')

churn_counts = df['Churn'].value_counts()
total = churn_counts.sum()
labels_with_counts = [f'{label}\n{count} ({count/total:.1%})' for label, count in zip(churn_counts.index, churn_counts)]
plt.pie(churn_counts, labels=labels_with_counts, autopct='%1.1f%%', startangle=140)
plt.title('Internet Service Distribution')
plt.show()

null_counts = df.isna().sum()
print("Null Counts:")
print(null_counts)

contract_counts = df['Contract'].value_counts()
contract_counts.plot.bar()
OnlineSecurity_counts = df['OnlineSecurity'].value_counts()
OnlineSecurity_counts.plot.bar()
Churn_counts = df['Churn'].value_counts()
Churn_counts.plot.bar()


# 03-DP

# Add any pre-processing steps

# Change numeric to binary

# Dropping specific columns
columns_to_drop = ['ZipCode','customerID','gender','SeniorCitizen','Partner']
df_cleaned = df.drop(columns=columns_to_drop)

# Define a function to check if a value is blank or null
def is_blank_or_null(value):
    return pd.isna(value) or (isinstance(value, str) and len(value.strip()) == 0)

# Filter the dataset based on the given conditions
df_cleaned = df_cleaned[
    ~df_cleaned.apply(lambda row: (
        is_blank_or_null(row['DeviceProtection']) or
        is_blank_or_null(row['TechSupport']) or
        pd.isna(row['TotalCharges']) or
        pd.isna(row['MonthlyCharges'])
    ), axis=1)
]
null_counts = df_cleaned.isna().sum()
print("Null Counts:")
print(null_counts)


# Correcting inconsistencies in relevant columns
df_cleaned['Churn'] = df_cleaned['Churn'].replace({1: 'Yes', 0: 'No'})
df_cleaned['OnlineSecurity'] = df_cleaned['OnlineSecurity'].replace({'Yes1': 'Yes', 'No2': 'No'})
df_cleaned['PaymentMethod'] = df_cleaned['PaymentMethod'].replace({
    'Electronic check111': 'Electronic check', 'Bank transfer (automatic)222': 'Bank transfer (automatic)'})
df_cleaned['Contract'] = df_cleaned['Contract'].replace({
    'One year3224': 'One year',
    'Two year23': 'Two year'
})

contract_counts = df_cleaned['Contract'].value_counts()
contract_counts.plot.bar()
OnlineSecurity_counts = df_cleaned['OnlineSecurity'].value_counts()
OnlineSecurity_counts.plot.bar()
Churn_counts = df_cleaned['Churn'].value_counts()
Churn_counts.plot.bar()
Churn_counts = df_cleaned['PaymentMethod'].value_counts()
Churn_counts.plot.bar()

# Determine the 10th and 90th percentile thresholds for `MonthlyCharges` and `TotalCharges`
monthly_charges_lower = df_cleaned['MonthlyCharges'].quantile(0.05)
monthly_charges_upper = df_cleaned['MonthlyCharges'].quantile(0.95)
total_charges_lower = df_cleaned['TotalCharges'].quantile(0.05)
total_charges_upper = df_cleaned['TotalCharges'].quantile(0.95)

# Filtering out rows outside these thresholds
df_cleaned = df_cleaned[
    (df_cleaned['MonthlyCharges'] >= monthly_charges_lower) & (df_cleaned['MonthlyCharges'] <= monthly_charges_upper) &
    (df_cleaned['TotalCharges'] >= total_charges_lower) & (df_cleaned['TotalCharges'] <= total_charges_upper)
]


# Define columns representing individual services
service_columns = [
    'InternetService', 'StreamingTV', 'StreamingMovies', 
    'TechSupport', 'DeviceProtection', 'OnlineBackup', 'OnlineSecurity'
]

# Converting these columns to binary indicators (1 if 'Yes', 0 if 'No')
for col in service_columns:
    df_cleaned[col] = df_cleaned[col].apply(lambda x: 1 if x == 'Yes' else 0)

# Create a new feature for total services used
df_cleaned['TotalServicesUsed'] = df_cleaned[service_columns].sum(axis=1)

# Define a function to group tenure
def tenure_group(tenure):
    if tenure <= 12:
        return 0
    elif tenure <= 24:
        return 1
    elif tenure <= 36:
        return 2
    elif tenure <= 48:
        return 3
    elif tenure <= 60:
        return 4
    else:
        return 5

# Apply the function to create a new column for tenure grouping
df_cleaned['TenureGroup'] = df_cleaned['tenure'].apply(tenure_group)

# Load the split datasets from Excel files
data_part1 = pd.read_excel('/Users/xm/Documents/UOA/info722/Iteration2_project_files/WA_Fn-UseC_-Telco-Customer-Churn-dirty.xlsx')
data_part2 = pd.read_excel('/Users/xm/Documents/UOA/info722/Iteration2_project_files/WA_Fn-UseC_-Telco-Customer-Churn-dirty2.xlsx')
merged_data = pd.concat([data_part1, data_part2], axis=1)

# Define label mapping for categorical columns
label_mapping = {
    'No': 0,
    'Yes': 1,
    'No internet service': 0,
    'DSL': 1,
    'Fiber optic': 2,
    'Electronic check': 1,
    'Mailed check': 2,
    'Bank transfer (automatic)': 3,
    'Credit card (automatic)': 4,
    'Month-to-month': 0,
    'One year': 1,
    'Two year': 2,
    'No phone service': 0  # Added mapping for MultipleLines
}

# Function to map categorical values to numerical labels if needed
def safe_map(value, mapping):
    return mapping.get(value, value) if isinstance(value, str) else value

# Apply safe_map to relevant columns
df_cleaned['Churn'] = df_cleaned['Churn'].apply(lambda x: safe_map(x, label_mapping))
df_cleaned['PaymentMethod'] = df_cleaned['PaymentMethod'].apply(lambda x: safe_map(x, label_mapping))
df_cleaned['Dependents'] = df_cleaned['Dependents'].apply(lambda x: safe_map(x, label_mapping))
df_cleaned['PhoneService'] = df_cleaned['PhoneService'].apply(lambda x: safe_map(x, label_mapping))
df_cleaned['MultipleLines'] = df_cleaned['MultipleLines'].apply(lambda x: safe_map(x, label_mapping))
df_cleaned['Contract'] = df_cleaned['Contract'].apply(lambda x: safe_map(x, label_mapping))
df_cleaned['PaperlessBilling'] = df_cleaned['PaperlessBilling'].apply(lambda x: safe_map(x, label_mapping))
df_cleaned.info()

# 04-DT

# Add any transformation steps

# One hot encoding for categorical
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns

# Selecting all relevant columns
categorical_features = ['InternetService', 'PaymentMethod', 'TenureGroup', 'StreamingTV', 
                        'StreamingMovies', 'TechSupport', 'DeviceProtection', 'OnlineBackup', 
                        'OnlineSecurity', 'Dependents', 'PhoneService', 'MultipleLines', 
                        'Contract', 'PaperlessBilling']
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges', 'TotalServicesUsed']

# Chi-square test for categorical features
chi2_vals, p_vals = chi2(pd.get_dummies(df_cleaned[categorical_features]), df_cleaned['Churn'])

# Display chi-square results
print("Chi-square results for categorical features:")
for feature, chi2_val, p_val in zip(categorical_features, chi2_vals, p_vals):
    print(f"Feature: {feature}, Chi2: {chi2_val:.4f}, P-value: {p_val:.4f}")

# Correlation matrix for numeric features
corr_matrix = df_cleaned[numeric_features + ['Churn']].corr()

# Display correlation matrix
print("Correlation matrix:\n", corr_matrix)

# Correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation Matrix")
plt.show()

# Train a model to get feature importance
X = df_cleaned.drop(columns=['Churn'])
y = df_cleaned['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Feature importances
feature_importances = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)

# Display feature importances
print("Feature Importances:\n", feature_importances)

columns_to_drop = ['Dependents', 'TotalServicesUsed','PhoneService', 'InternetService']
df_cleaned = df_cleaned.drop(columns=columns_to_drop)

import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer

# Log transformation for skewed features
df_cleaned['LogTotalCharges'] = np.log1p(df_cleaned['TotalCharges'])  # log1p to handle zeros
df_cleaned['LogMonthlyCharges'] = np.log1p(df_cleaned['MonthlyCharges'])

# Applying Box-Cox transformation
pt = PowerTransformer(method='box-cox')
df_cleaned[['LogTotalCharges', 'LogMonthlyCharges']] = pt.fit_transform(df_cleaned[['LogTotalCharges', 'LogMonthlyCharges']])

df_cleaned['LogMonthlyCharges'].plot.hist(bins=50)
plt.xlabel('LogMonthlyCharges Charges')
plt.ylabel('Frequency')
plt.title('Distribution of LogMonthlyCharges')

df_cleaned['LogTotalCharges'].plot.hist(bins=50)
plt.xlabel('LogTotalCharges')
plt.ylabel('Frequency')
plt.title('Distribution of LogTotalCharges')

# Retain original numerical features and create standardized versions
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
standardized_numeric_features = ['Std_' + feat for feat in numeric_features]

scaler = StandardScaler()
df_cleaned[standardized_numeric_features] = scaler.fit_transform(df_cleaned[numeric_features])

# 05-DMM

"""

Identify the Data Mining method
Describe how it aligns with the objectives

"""

# 06-DMA

# Load relevant algorithms
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.tree import plot_tree

# Prepare features for modeling
X = df_cleaned.drop(columns=['Churn'])
y = df_cleaned['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


    
# 07-DM

# Execute DM task

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
# Evaluate Different Splits
splits = [(0.7, 0.3), (0.8, 0.2), (0.9, 0.1)]

for train_size, test_size in splits:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Train model
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = rf.predict(X_test)
    
    print(f"Split: {int(train_size * 100)}/{int(test_size * 100)}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred))
    print("ROC-AUC:", roc_auc_score(y_test, y_pred))
    print()

# Random Forest model
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
# Feature Importance
feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
# Visualize Feature Importance
feature_importances.plot(kind='bar', title="Random Forest Feature Importance")
plt.show()
# Visualize a Specific Tree
plt.figure(figsize=(12, 8))
plot_tree(rf.estimators_[0], feature_names=X.columns, class_names=['No Churn', 'Churn'], filled=True)
plt.title("Random Forest Tree Visualization")
plt.show()


# Neural Network model
nn = MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu', 
                   learning_rate_init=0.01, max_iter=200, random_state=42)
nn.fit(X_train, y_train)
# Extract weights from the model
input_to_hidden_weights = nn.coefs_[0]  # Weights from input to first hidden layer
# Visualize the weights
feature_influence = pd.DataFrame(input_to_hidden_weights, index=X.columns)
feature_influence.sum(axis=1).sort_values(ascending=False).plot(kind='bar', title="Feature Influence from NN Weights")
plt.show()
# Heatmap for Weights
sns.heatmap(input_to_hidden_weights, cmap='coolwarm')
plt.title("Neural Network Weights (Input to Hidden Layer)")
plt.show()


# XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, 
                              max_depth=10, random_state=42)
xgb_model.fit(X_train, y_train)
# Feature Importance
importances = xgb_model.get_booster().get_score(importance_type='weight')
# Convert to a pandas Series
importances_series = pd.Series(importances).sort_values(ascending=False)
print("XGBoost Feature Importance (by weight):\n", importances_series)
# Visualize the feature importance
importances_series.plot(kind='bar', title="XGBoost Feature Importance (Weight)")
plt.show()

# Logistic Regression model
log_reg = LogisticRegression(max_iter=500, solver='liblinear', 
                             C=1.0, penalty='l2', random_state=42)
log_reg.fit(X_train, y_train)
# Coefficients
coefficients = pd.Series(log_reg.coef_[0], index=X.columns)
print("Logistic Regression Coefficients:\n", coefficients)
# Visualize Coefficients
coefficients.plot(kind='bar', title="Logistic Regression Coefficients")
plt.show()

# KNN Model
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')  # k can be adjusted
knn.fit(X_train, y_train)
# Predict Churn
y_pred = knn.predict(X_test)
# Scatter Plot for Continuous Features
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['Churn'] = y_test
for feature in numeric_features:
    plt.figure(figsize=(8, 6))
    plt.scatter(df_test[feature], df_test['Churn'], c=y_pred, cmap='coolwarm')
    plt.xlabel(feature)
    plt.ylabel('Churn')
    plt.title(f"Scatter Plot: {feature} vs Churn")
    plt.show()

    
# StratifiedKFold for balanced splits
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Models
models = {
    "Random Forest": rf,
    "Logistic Regression": log_reg,
    "Neural Networks": nn,
    "XGBoost": xgb_model,
    "KNN":knn
}

# Evaluating models
for name, model in models.items():
    print(f"Evaluating {name} Model:")

    # Cross-validation scores
    acc_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    prec_scores = cross_val_score(model, X, y, cv=cv, scoring='precision')
    rec_scores = cross_val_score(model, X, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(model, X, y, cv=cv, scoring='f1')
    
    print(f"Accuracy: {np.mean(acc_scores):.4f}")
    print(f"Precision: {np.mean(prec_scores):.4f}")
    print(f"Recall: {np.mean(rec_scores):.4f}")
    print(f"F1 Score: {np.mean(f1_scores):.4f}")
    print()

# 08-INT

# Summarise Results

# Add relevant tables or graphs
from sklearn.metrics import confusion_matrix
import shap

# Histograms
plt.figure(figsize=(10, 6))
plt.hist(df_cleaned['TotalCharges'], bins=30, alpha=0.7, label='TotalCharges')
plt.hist(df_cleaned['MonthlyCharges'], bins=30, alpha=0.7, label='MonthlyCharges')
plt.legend()
plt.title("Distribution of Total and Monthly Charges")
plt.show()


# Confusion Matrix
# Evaluation
y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Scatter Plot for Continuous Features
df_test = pd.DataFrame(X_test, columns=X.columns)
df_test['Churn'] = y_test

for feature in numeric_features:
    plt.figure(figsize=(8, 6))
    plt.scatter(df_test[feature], df_test['Churn'], c=df_test['Churn'], cmap='coolwarm')
    plt.xlabel(feature)
    plt.ylabel('Churn')
    plt.title(f"Scatter Plot: {feature} vs Churn")
    plt.show()

# Learning Curves
training_loss = nn.loss_curve_

plt.figure(figsize=(10, 6))
plt.plot(training_loss, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Learning Curve")
plt.legend()
plt.show()

# Feature Importance
feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
feature_importances.plot(kind='bar', title="Random Forest Feature Importance")
plt.show()

# Correlation Matrix
corr_matrix = df_cleaned.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation Matrix")
plt.show()

# Evaluation Metrics
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    
    results.append({"Model": name, "Accuracy": accuracy, "F1 Score": f1, "ROC-AUC": roc_auc})

# Convert to a DataFrame
results_df = pd.DataFrame(results)

# Visualize the Results
plt.figure(figsize=(10, 6))
results_df.set_index("Model")[["Accuracy", "F1 Score", "ROC-AUC"]].plot(kind='bar')
plt.title("Model Performance Comparison")
plt.show()

# SHAP Analysis
background_sample = shap.kmeans(X_train, 20)
explainer = shap.KernelExplainer(rf.predict_proba, background_sample)
shap_values = explainer.shap_values(X_test)

print("SHAP values shape:", shap_values[1].shape)
print("X_test shape:", X_test.shape)
# SHAP Summary Plot
shap.summary_plot(shap_values[1], X_test, feature_names=X.columns)

# SHAP Dependency Plot
shap.dependence_plot("TotalCharges", shap_values[1], X_test, feature_names=X.columns)

# 09-ACT

"""

Desribe the Action Plan to Implement, Observe and Improve

"""


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.